In [ ]:
# ===================================================================
# 1️⃣ 단계: 라이브러리 설치 및 임포트
# ===================================================================
!pip install xgboost

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import recall_score
from sklearn.ensemble import StackingClassifier, RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
import warnings
import contextlib
import io

warnings.filterwarnings('ignore')


# ===================================================================
# 2️⃣ 단계: 데이터 로드 및 설정
# ===================================================================
# ⚠️ 중요: 아래 'TARGET_COLUMN' 변수 값을 실제 타겟 컬럼명으로 수정해주세요!
# 예: TARGET_COLUMN = '부도'
TARGET_COLUMN = '여기에_타겟변수_컬럼이름을_입력하세요'

file_path = '/content/final_부동산.csv'

try:
    df = pd.read_csv(file_path)
    print(f"✅ '{file_path}' 파일 로드 성공!")
except FileNotFoundError:
    print(f"❌ 오류: '{file_path}' 경로에 파일이 없습니다. 파일을 먼저 업로드해주세요.")
    # 파일이 없으면 더 이상 진행하지 않음
    exit()

# 초기 20개 피처 리스트 (타겟 변수는 이 리스트에 포함되지 않아야 함)
initial_features = [
    '연체과목수_3개월유지', '연체기관수_전체', '최장연체일수_3개월',
    '최장연체일수_6개월', '최장연체일수_1년', '최장연체일수_3년', '연체경험',
    '유동자산', '비유동자산', '자산총계', '유동부채', '비유동부채',
    '부채총계', '매출액', '매출총이익', '영업손익', '당기순이익', '영업활동현금흐름',
    '재무비율_부채비율', '재무비율_유동비율'
]

# 타겟 변수와 피처 분리
if TARGET_COLUMN not in df.columns:
    print(f"❌ 오류: 데이터에 '{TARGET_COLUMN}' 컬럼이 없습니다. 컬럼명을 확인하고 수정해주세요.")
    print("사용 가능한 컬럼:", df.columns.tolist())
    exit()

X = df[initial_features]
y = df[TARGET_COLUMN]

# 훈련/테스트 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)


# ===================================================================
# 3️⃣ 단계: 파이프라인 및 후진 제거법 함수 정의
# ===================================================================
class BankruptcyPredictionPipeline:
    """기업부도 예측 파이프라인"""
    def __init__(self):
        self.model = None
        self.feature_names = None
        self.is_fitted = False
        self.selected_features = []

    def fit(self, X: pd.DataFrame, y: pd.Series):
        self.feature_names = self.selected_features
        X_selected = X[self.feature_names]
        estimators = [
            ('xgb', XGBClassifier(random_state=42, use_label_encoder=False, eval_metric='logloss')),
            ('rf', RandomForestClassifier(random_state=42, class_weight='balanced'))
        ]
        self.model = StackingClassifier(estimators=estimators, final_estimator=LogisticRegression(class_weight='balanced'), cv=3, n_jobs=-1)
        self.model.fit(X_selected, y)
        self.is_fitted = True
        return self

    def predict(self, X: pd.DataFrame) -> np.ndarray:
        X_selected = X[self.feature_names].copy()
        return self.model.predict(X_selected)

def run_backward_elimination(X_train, y_train, X_test, y_test, initial_features, recall_threshold=0.79):
    """후진 제거법을 수행하여 최적의 피처 조합을 찾습니다."""
    current_features = initial_features.copy()
    print(f"🚀 후진 제거법 시작... (Recall 유지 조건: {recall_threshold} 이상)")
    print("-" * 60)
    step = 1
    while len(current_features) > 1:
        recall_per_feature = []
        for feature_to_remove in current_features:
            temp_features = [f for f in current_features if f != feature_to_remove]
            pipeline = BankruptcyPredictionPipeline()
            pipeline.selected_features = temp_features
            with contextlib.redirect_stdout(io.StringIO()):
                pipeline.fit(X_train, y_train)
            y_pred = pipeline.predict(X_test)
            recall = recall_score(y_test, y_pred, pos_label=1)
            recall_per_feature.append({'feature': feature_to_remove, 'recall': recall})
        
        best_candidate = max(recall_per_feature, key=lambda x: x['recall'])
        
        if best_candidate['recall'] >= recall_threshold:
            feature_to_eliminate = best_candidate['feature']
            recall_at_elimination = best_candidate['recall']
            print(f"➡️ 단계 {step}: '{feature_to_eliminate}' 제거 (제거 후 Recall: {recall_at_elimination:.4f})")
            current_features.remove(feature_to_eliminate)
            step += 1
        else:
            print(f"\n⚠️ 중단: '{best_candidate['feature']}' 제거 시 Recall({best_candidate['recall']:.4f})이 임계값 미만.")
            break
            
    print("-" * 60)
    print("✅ 후진 제거법 완료!")
    return current_features

# ===================================================================
# 4️⃣ 단계: 실행 및 결과 출력
# ===================================================================
final_features = run_backward_elimination(X_train, y_test, X_test, y_test, initial_features)

print("\n\n--- 최종 결과 ---")
print(f"\n📊 최종 선택된 피처 개수: {len(final_features)}")
print("\n📋 최종 피처 목록:")
for feature in final_features:
    print(f"  - {feature}")